<a href="https://colab.research.google.com/github/marquesgabi/Doutorado/blob/master/Calculo_024_NOV_07_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# carrega as bibliotecas do python

In [0]:
# rotate image....
# https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/
# https://pythontic.com/image-processing/pillow/rotate

In [0]:
%matplotlib inline
from matplotlib import pyplot as plt
import os 
import cv2
import zipfile
from skimage import data
from skimage import filters
from skimage import exposure
import skimage.feature as sk
import numpy as np
import pandas as pd
from google.colab import files
from sklearn.metrics.cluster import entropy
from sklearn.metrics.cluster import homogeneity_score
from numpy import linalg as LA


In [0]:
!wget http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
#!ls -l trash_nov_22_2018.zip


--2019-11-08 12:54:08--  http://ftp.feq.ufu.br/ubirajara/Sugar_set_30_2019.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19584173 (19M) [application/zip]
Saving to: ‘Sugar_set_30_2019.zip.1’

Sugar_set_30_2019.z 100%[===================>]  18.68M  5.02MB/s    in 3.7s    

2019-11-08 12:54:12 (5.02 MB/s) - ‘Sugar_set_30_2019.zip.1’ saved [19584173/19584173]



In [0]:

file_name = zipfile.ZipFile('Sugar_set_30_2019.zip', 'r')
file_name.extractall() 

In [0]:
# all images are resized in variable x, scale gray in variable w and np.array in variable p para cada foto
FILE='Sugar_set_30_2019.zip'
img_name=[]
x=[]
p=[]
w=[]
Size=5 # trocar por 64 qdo acabar a analise

with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)
        #x.append(cv2.imread(name))
        x.append(cv2.resize(cv2.imread(name),(Size,Size)))

nrow=len(img_name)
ncol=Size*Size
pp=np.zeros((nrow,ncol)) # np.array pp é uma matriz e cada linha é uma foto
for i in range(nrow):
    w.append(cv2.cvtColor(np.array(x[i]), cv2.COLOR_BGR2GRAY))
    pp[i,:]=w[i].ravel()

In [0]:
!git clone https://github.com/ucfilho/marquesgabi #clonar do Github
%cd marquesgabi
import Go2GLCM

fatal: destination path 'marquesgabi' already exists and is not an empty directory.
/content/marquesgabi


In [0]:
Nfotos=len(img_name)
GLCM=[]
for k in range(Nfotos):
  p=pp[k,:].reshape(Size,Size) #voltar o pp para p novamente no formato ij, mas sem as infos estranhas
  MAT=Go2GLCM.GLCM(p)
  pd.set_option('display.max_columns', 30)
  GLCM.append(MAT.ravel())

Nomes=['ASM','constrast','correl','variance','inv diff mom','sum aveg',
       'sum var','sum entropy','entropy','dif var','dif entropy','IMC1',
       'IMC2','MCC']
#print(pd.DataFrame(GLCM,columns=Nomes))
print(pd.DataFrame(GLCM,columns=Nomes).head())

  

        ASM  constrast     correl  variance  inv diff mom   sum aveg  \
0  0.040159   4.006764 -48.388893  5.873156      0.423837  20.097900   
1  0.041540   4.282528 -47.724904  5.762803      0.411868  20.118283   
2  0.040287   4.030399 -49.193342  5.979967      0.424910  20.208554   
3  0.040175   4.015663 -48.607804  6.027608      0.435603  20.231465   
4  0.040313   4.043838 -47.746904  5.656749      0.422210  19.935322   

     sum var  sum entropy   entropy    dif var  dif entropy      IMC1  \
0  38.848134     2.080038  0.648663  20.033820     7.608990 -0.000770   
1  38.570089     2.064334  0.646616  21.412639     7.688757 -0.005531   
2  39.133866     2.071191  0.656965  20.151997     7.628194 -0.001239   
3  39.597120     2.083742  0.653877  20.078315     7.654684 -0.000658   
4  38.085237     2.073301  0.652921  20.219188     7.614789 -0.001073   

       IMC2       MCC  
0  0.049748  0.039787  
1  0.132720  0.133372  
2  0.063068  0.045709  
3  0.045987  0.036992  
4  0.058

In [0]:
print(p.shape)

(5, 5)


In [0]:
'''          
          0         1          2         3         4          5          6  \
0  0.040159  4.006764 -48.388893  5.873156  0.423837  20.097900  38.848134   
1  0.041540  4.282528 -47.724904  5.762803  0.411868  20.118283  38.570089   
2  0.040287  4.030399 -49.193342  5.979967  0.424910  20.208554  39.133866   
3  0.040175  4.015663 -48.607804  6.027608  0.435603  20.231465  39.597120   
4  0.040313  4.043838 -47.746904  5.656749  0.422210  19.935322  38.085237   

          7         8          9        10        11        12        13  
0  2.080038  0.648663  20.033820  7.608990 -0.000770  0.049748  0.039787  
1  2.064334  0.646616  21.412639  7.688757 -0.005531  0.132720  0.133372  
2  2.071191  0.656965  20.151997  7.628194 -0.001239  0.063068  0.045709  
3  2.083742  0.653877  20.078315  7.654684 -0.000658  0.045987  0.036992  
4  2.073301  0.652921  20.219188  7.614789 -0.001073  0.058718  0.051329
'''

'          \n          0         1          2         3         4          5          6  0  0.040159  4.006764 -48.388893  5.873156  0.423837  20.097900  38.848134   \n1  0.041540  4.282528 -47.724904  5.762803  0.411868  20.118283  38.570089   \n2  0.040287  4.030399 -49.193342  5.979967  0.424910  20.208554  39.133866   \n3  0.040175  4.015663 -48.607804  6.027608  0.435603  20.231465  39.597120   \n4  0.040313  4.043838 -47.746904  5.656749  0.422210  19.935322  38.085237   \n\n          7         8          9        10        11        12        13  \n0  2.080038  0.648663  20.033820  7.608990 -0.000770  0.049748  0.039787  \n1  2.064334  0.646616  21.412639  7.688757 -0.005531  0.132720  0.133372  \n2  2.071191  0.656965  20.151997  7.628194 -0.001239  0.063068  0.045709  \n3  2.083742  0.653877  20.078315  7.654684 -0.000658  0.045987  0.036992  \n4  2.073301  0.652921  20.219188  7.614789 -0.001073  0.058718  0.051329\n'

In [0]:
import numpy as np
#from skimage.feature import haar_like_feature_coord
#from skimage.feature import draw_haar_like_feature
from skimage.feature import greycomatrix
from skimage.feature import greycoprops
#import skimage.feature.greycoprops as greycoprops
image = np.array([[0, 0, 1, 1],[0, 0, 1, 1],[0, 2, 2, 2],[2, 2, 3, 3]], dtype=np.uint8)
g = greycomatrix(image, [1, 2], [0, np.pi/2], levels=6,normed=True, symmetric=True)
contrast = greycoprops(g, 'contrast')
correlation= greycoprops(g, 'correlation')
print(contrast)
print(correlation)

[[0.58333333 1.        ]
 [1.25       2.75      ]]
[[ 0.71953255  0.48571429]
 [ 0.41176471 -0.29411765]]


In [0]:
image = np.array(w[0], dtype=np.uint8)
g = greycomatrix(image, [1, 2], [0, np.pi/2], levels=255,normed=True, symmetric=True)
contrast = greycoprops(g, 'contrast')
correlation= greycoprops(g, 'correlation')
print(contrast)
print(correlation)

[[ 98.2         85.1       ]
 [153.66666667  72.4       ]]
[[-0.0829887   0.15825915]
 [-0.56657076  0.20068695]]


In [0]:
P=w[0]
print
row,col=P.shape
Size=row
Soma=0
for i in range(Size):
    for j in range(Size):
      Soma=Soma+P[i,j]
  
for i in range(Size):
  for j in range(Size):
    P[i,j]=P[i,j]/Soma

image = np.array(P, dtype=np.uint8)
g = greycomatrix(image, [1, 2], [0, np.pi/2], levels=255,normed=True, symmetric=True)
contrast = greycoprops(g, 'contrast')
correlation= greycoprops(g, 'correlation')
print(contrast)
print(correlation)

[[0. 0.]
 [0. 0.]]
[[1. 1.]
 [1. 1.]]


In [0]:
print(P);print(Size)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
5


In [0]:
# glcm = [0 1 2 3;1 1 2 3;1 0 2 0;0 0 0 3]

In [0]:
'''
https://www.mathworks.com/help/images/ref/graycoprops.html
glcm = 4×4

     0     1     2     3
     1     1     2     3
     1     0     2     0
     0     0     0     3
'''

'\nhttps://www.mathworks.com/help/images/ref/graycoprops.html\nglcm = 4×4\n\n     0     1     2     3\n     1     1     2     3\n     1     0     2     0\n     0     0     0     3\n'

In [0]:
'''
stats = graycoprops(glcm)
stats = struct with fields:
       Contrast: 2.8947
    Correlation: 0.0783
         Energy: 0.1191
    Homogeneity: 0.5658
'''

'\nstats = graycoprops(glcm)\nstats = struct with fields:\n       Contrast: 2.8947\n    Correlation: 0.0783\n         Energy: 0.1191\n    Homogeneity: 0.5658\n'

In [0]:
glcm = np.array([[0, 1, 2, 3],[1, 1, 2, 3],[1, 0, 2, 0],[0, 0, 0, 3]]);print(glcm)

[[0 1 2 3]
 [1 1 2 3]
 [1 0 2 0]
 [0 0 0 3]]


In [0]:
glcm = np.array([[0, 1, 2, 3],[1, 1, 2, 3],[1, 0, 2, 0],[0, 0, 0, 3]]);print(glcm)

[[0 1 2 3]
 [1 1 2 3]
 [1 0 2 0]
 [0 0 0 3]]


In [0]:
image = np.array(glcm, dtype=np.uint8)
g = greycomatrix(image, [0,1], [0, np.pi/2], levels=4, normed=True, symmetric=True)
contrast = greycoprops(g, 'contrast')
correlation= greycoprops(g, 'correlation')
print(contrast)
print(correlation)

[[0.         0.        ]
 [1.91666667 2.08333333]]
[[1.         1.        ]
 [0.13615023 0.10581222]]


In [0]:
Glcm_test = np.array([[0, 1, 2, 3],[1, 1, 2, 3],[1, 0, 2, 0],[0, 0, 0, 3]]);print(Glcm_test)

[[0 1 2 3]
 [1 1 2 3]
 [1 0 2 0]
 [0 0 0 3]]


In [0]:
row,col=glcm.shape
print('====');print(Glcm_test)
GLCMat=np.zeros((row,col))
for i in range(row):
  for j in range(col):
    print(Glcm_test[i,j])
    A=2**Glcm_test[i,j]
    GLCMat[i,j]=A
print(GLCMat)
MAT=Go2GLCM.GLCM(GLCMat) # como calcular se tem zero???
Nomes=['ASM','constrast','correl','variance','inv diff mom','sum aveg',
       'sum var','sum entropy','entropy','dif var','dif entropy','IMC1',
       'IMC2','MCC']
print(pd.DataFrame(MAT,columns=Nomes).head())
#print(MAT)

====
[[0 1 2 3]
 [1 1 2 3]
 [1 0 2 0]
 [0 0 0 3]]
0
1
2
3
1
1
2
3
1
0
2
0
0
0
0
3
[[1. 2. 4. 8.]
 [2. 2. 4. 8.]
 [2. 1. 4. 1.]
 [1. 1. 1. 8.]]
      ASM  constrast     correl  variance  inv diff mom  sum aveg    sum var  \
0  0.1048       3.04 -35.910537  2.781406         0.488     13.76  21.386099   

   sum entropy  entropy  dif var  dif entropy      IMC1      IMC2       MCC  
0     1.702149  0.95788    12.16     5.456622 -0.064736  0.400743  0.388483  


In [0]:
print(MAT[0,0]**0.5)

0.3237282811247729


In [0]:
glcm = np.array([[0, 1, 2, 3],[1, 1, 2, 3],[1, 0, 2, 0],[0, 0, 0, 3]]);
print(glcm)

[[0 1 2 3]
 [1 1 2 3]
 [1 0 2 0]
 [0 0 0 3]]


In [0]:
MAT=Go2GLCM.GLCM(glcm) # como calcular se tem zero???
print(MAT)

/content/marquesgabi/Go2GLCM.py:72: RuntimeWarning: divide by zero encountered in log
  HXY=HXY-p[i,j]*np.log(p[i,j])
/content/marquesgabi/Go2GLCM.py:72: RuntimeWarning: invalid value encountered in multiply
  HXY=HXY-p[i,j]*np.log(p[i,j])
/content/marquesgabi/Go2GLCM.py:73: RuntimeWarning: divide by zero encountered in log
  HXY1=HXY1-p[i,j]*np.log(px[i]*py[j])
/content/marquesgabi/Go2GLCM.py:73: RuntimeWarning: invalid value encountered in multiply
  HXY1=HXY1-p[i,j]*np.log(px[i]*py[j])
/content/marquesgabi/Go2GLCM.py:74: RuntimeWarning: divide by zero encountered in log
  HXY2=HXY2-px[i]*py[j]*np.log(px[i]*py[j])
/content/marquesgabi/Go2GLCM.py:74: RuntimeWarning: invalid value encountered in multiply
  HXY2=HXY2-px[i]*py[j]*np.log(px[i]*py[j])
/content/marquesgabi/Go2GLCM.py:77: RuntimeWarning: divide by zero encountered in log
  HX=HX-px[i]*np.log(px[i])
/content/marquesgabi/Go2GLCM.py:77: RuntimeWarning: invalid value encountered in multiply
  HX=HX-px[i]*np.log(px[i])
/content/m

LinAlgError: ignored